# Lake model continued - subspace partitioning

In the previous week you used the lake problem as a means of getting acquainted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of the [open exploration tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the seperate anthropogenic release decision**

## 1. Apply scenario discovery

1. Instanciate the model and define its parameters. Use the same parameters as in Assignment 2.
2. Generate 10 policies and 1000 scenarios and evaluate them.
3. The experiments array contains the values for each of the 100 decision levers. This might easily mess up the analysis. Remove these columns from the experiment array. *hint: use `experiments.drop_columns`*
4. Apply scenario discovery, focussing on the 10 percent of worst outcomes for reliability

In [1]:
from lakemodel_function import lake_problem
from ema_workbench import Samplers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import Model, RealParameter, TimeSeriesOutcome, ScalarOutcome, SequentialEvaluator
from scipy.integrate import odeint


In [2]:
# Instantiate the model
model = Model('lake', function=lake_problem)
time_horizon = 100

# Specify uncertainties  # Dit is dus een vijf dimensionele space van uncertainties. Met voor elke dimensie een min en max waarde zeg maar.
model.uncertainties = [RealParameter('mean',0.01,0.05), RealParameter('stdev',0.001,0.005),
                       RealParameter('b',0.1,0.45), RealParameter('q',2,4.5),
                       RealParameter('delta',0.93,0.99)]

# Set levers, one for each time step    # Dit is dus een 100 dimensionele space van uncertainties. Met voor elke dimensie dezelfde min en max waarde.
# Dit geeft aan hoeveel schadelijke vloeistof elke tijdstip gedumpt mag worden in de rivier als beleid. Hier kan mee geëxperimenteerd worden.
levers = []
for i in range(time_horizon):
    lever = RealParameter(f'l{i}',0.05,0.1)
    levers.append(lever)
#print(levers)
model.levers = levers

# Specify outcomes
model.outcomes = [ScalarOutcome('max_P'), ScalarOutcome('utility'),
                  ScalarOutcome('inertia'), ScalarOutcome('reliability')]


#Waarom is dit Scalar en geen Time Series?

In [3]:
from ema_workbench import SequentialEvaluator
from ema_workbench import ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)


with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=1000, policies=10)

[MainProcess/INFO] performing 1000 scenarios * 10 policies * 1 model(s) = 10000 experiments
100%|███████████████████████████████████| 10000/10000 [01:36<00:00, 103.22it/s]
[MainProcess/INFO] experiments finished


In [4]:
experiments

,b,delta,mean,q,stdev,l0,l1,l10,l11,l12,...,l93,l94,l95,l96,l97,l98,l99,scenario,policy,model
0,0.283023,0.935505,0.024144,2.352048,0.004608,0.061341,0.080724,0.088167,0.072309,0.081942,...,0.057793,0.097825,0.095845,0.069363,0.093703,0.060337,0.081481,10,0,lake
1,0.264206,0.961968,0.022775,2.042400,0.001581,0.061341,0.080724,0.088167,0.072309,0.081942,...,0.057793,0.097825,0.095845,0.069363,0.093703,0.060337,0.081481,11,0,lake
2,0.374184,0.967813,0.047590,4.075519,0.003891,0.061341,0.080724,0.088167,0.072309,0.081942,...,0.057793,0.097825,0.095845,0.069363,0.093703,0.060337,0.081481,12,0,lake
3,0.278220,0.962170,0.024279,3.782230,0.002469,0.061341,0.080724,0.088167,0.072309,0.081942,...,0.057793,0.097825,0.095845,0.069363,0.093703,0.060337,0.081481,13,0,lake
4,0.299137,0.969146,0.031180,4.340810,0.004656,0.061341,0.080724,0.088167,0.072309,0.081942,...,0.057793,0.097825,0.095845,0.069363,0.093703,0.060337,0.081481,14,0,lake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.395180,0.961266,0.042110,3.876651,0.003557,0.065444,0.097118,0.076491,0.087776,0.096256,...,0.061503,0.081366,0.062651,0.053357,0.097811,0.056660,0.064734,1005,9,lake
9996,0.417669,0.974647,0.046543,3.907393,0.004190,0.065444,0.097118,0.076491,0.087776,0.096256,...,0.061503,0.081366,0.062651,0.053357,0.097811,0.056660,0.064734,1006,9,lake
9997,0.227694,0.981036,0.032638,2.876067,0.001963,0.065444,0.097118,0.076491,0.087776,0.096256,...,0.061503,0.081366,0.062651,0.053357,0.097811,0.056660,0.064734,1007,9,lake
9998,0.163218,0.981754,0.017523,3.221174,0.001607,0.065444,0.097118,0.076491,0.087776,0.096256,...,0.061503,0.081366,0.062651,0.053357,0.097811,0.056660,0.064734,1008,9,lake


In [9]:
cleaned_experiments = experiments.drop(labels=[l.name for l in model.levers], axis=1)
cleaned_experiments

,b,delta,mean,q,stdev,scenario,policy,model
0,0.283023,0.935505,0.024144,2.352048,0.004608,10,0,lake
1,0.264206,0.961968,0.022775,2.042400,0.001581,11,0,lake
2,0.374184,0.967813,0.047590,4.075519,0.003891,12,0,lake
3,0.278220,0.962170,0.024279,3.782230,0.002469,13,0,lake
4,0.299137,0.969146,0.031180,4.340810,0.004656,14,0,lake
...,...,...,...,...,...,...,...,...
9995,0.395180,0.961266,0.042110,3.876651,0.003557,1005,9,lake
9996,0.417669,0.974647,0.046543,3.907393,0.004190,1006,9,lake
9997,0.227694,0.981036,0.032638,2.876067,0.001963,1007,9,lake
9998,0.163218,0.981754,0.017523,3.221174,0.001607,1008,9,lake


In [7]:
data = outcomes['reliability']
np.percentile(data,10)



0.04

In [ ]:
y = data < np.percentile(data, 10)
y


In [ ]:
from ema_workbench.analysis import prim
prim_alg = prim.Prim(cleaned_experiments,y, threshold=0.8)
box1 = prim_alg.find_box()

## 2. Visualize the results using Dimensional Stacking
Take the classification of outcomes as used in step 3 of scenario discovery, and instead visualize the results using dimensional stacking. How do these results compare to the insights from scenario discovery?